## LightGCN

pure collaborative filtering LightGCN

In [1]:
import numpy as np
import pandas as pd
from libreco.data import random_split, DatasetPure
from libreco.algorithms import LightGCN  # pure data, algorithm LightGCN
from libreco.evaluation import evaluate

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
df = pd.read_csv("cf_initial.csv")
df= df.rename(columns={"userID":"user", 'itemID':"item", "rating":"label", "song_by": "song"})
df=df[["user","item","label","song"]]
df.nunique()

user       861
item     58320
label        1
song     56714
dtype: int64

In [3]:
# split whole data into three folds for training, evaluating and testing
train_data, eval_data, test_data = random_split(df, multi_ratios=[0.8, 0.1, 0.1])

train_data, data_info = DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)
print(data_info)

n_users: 861, n_items: 48529, data density: 0.1596 %


In [11]:
lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="bpr",
    embed_size=16,
    n_epochs=2,
    lr=1e-3,
    batch_size=50,
    num_neg=1,
    device="cuda",
)

In [27]:
import optuna

def objective(trial):
    # Define hyperparameter search space
    lr = trial.suggest_float("lr", 1e-5, 1e-2)
    batch_size = trial.suggest_int("batch_size", 128, 256)
    embed_size = trial.suggest_int("embed_size", 8, 256)
    num_epochs = trial.suggest_int("num_epochs", 2, 8)

    # Create and train the model with the suggested hyperparameters
    lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="bpr",
    embed_size=embed_size,
    n_epochs=num_epochs,
    lr=lr,
    batch_size=batch_size,
    num_neg=1,
    device="cuda",
)
    lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)

    error = evaluate(
        model=lightgcn,
        data=test_data,
        neg_sampling=True,
        metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
    )
    return error['precision']

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Get the best hyperparameters from the study
best_params = study.best_params

print(best_params)


[I 2023-11-15 00:28:09,305] A new study created in memory with name: no-name-6baae4ce-8ffe-4dd2-a2ff-8d86f164d185


Training start time: 2023-11-15 00:28:09


train: 100%|██████████| 282/282 [02:44<00:00,  1.71it/s]


Epoch 1 elapsed: 164.766s
	 train_loss: 0.0779


eval_listwise: 100%|██████████| 679/679 [00:01<00:00, 510.99it/s]


...

Epoch 5 elapsed: 114.115s
	 train_loss: 0.004


eval_listwise: 100%|██████████| 679/679 [00:01<00:00, 658.95it/s]


	 eval log_loss: 0.5496
	 eval roc_auc: 0.9462
	 eval precision@10: 0.0364
	 eval recall@10: 0.0621
	 eval ndcg@10: 0.1448


eval_listwise: 100%|██████████| 689/689 [00:01<00:00, 676.74it/s]
[I 2023-11-15 03:07:55,947] Trial 9 finished with value: 0.036284470246734396 and parameters: {'lr': 0.006874727667973322, 'batch_size': 146, 'embed_size': 53, 'num_epochs': 5}. Best is trial 9 with value: 0.036284470246734396.


{'lr': 0.006874727667973322, 'batch_size': 146, 'embed_size': 53, 'num_epochs': 5}


In [37]:
lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="bpr",
    embed_size=best_params['embed_size'],
    n_epochs=best_params['num_epochs'],
    lr=best_params['lr'],
    batch_size=best_params['batch_size'],
    num_neg=1,
    device="cuda",
)

In [38]:
# monitor metrics on eval data during training
lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)

Training start time: 2023-11-15 12:13:00


train: 100%|██████████| 457/457 [02:09<00:00,  3.53it/s]


Epoch 1 elapsed: 129.393s
	 train_loss: 0.0939


eval_listwise: 100%|██████████| 679/679 [00:01<00:00, 625.38it/s]


	 eval log_loss: 0.4993
	 eval roc_auc: 0.9500
	 eval precision@10: 0.0349
	 eval recall@10: 0.0634
	 eval ndcg@10: 0.1376


...

	 eval log_loss: 0.5402
	 eval roc_auc: 0.9477
	 eval precision@10: 0.0355
	 eval recall@10: 0.0615
	 eval ndcg@10: 0.1432


train: 100%|██████████| 457/457 [01:54<00:00,  3.99it/s]


Epoch 5 elapsed: 114.561s
	 train_loss: 0.004


eval_listwise: 100%|██████████| 679/679 [00:01<00:00, 657.07it/s]

	 eval log_loss: 0.5496
	 eval roc_auc: 0.9462
	 eval precision@10: 0.0364
	 eval recall@10: 0.0621
	 eval ndcg@10: 0.1448


In [39]:
# do final evaluation on test data
evaluate(
    model=lightgcn,
    data=test_data,
    neg_sampling=True,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)

eval_listwise: 100%|██████████| 689/689 [00:01<00:00, 681.81it/s]


{'loss': 0.5726528606170258,
 'roc_auc': 0.9392017125025194,
 'precision': 0.036284470246734396,
 'recall': 0.06498846057989743,
 'ndcg': 0.1354837731009233}

### Prediction and Evaluation

In [71]:
# importing the evaluation metrics
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
                                                     recall_at_k, get_top_k_items)

In [46]:
# top k items to recommend
TOP_K = 10

In [61]:
train, eval, test = random_split(df, multi_ratios=[0.8, 0.1, 0.1])

In [82]:
users, items, preds = [], [], []
item = list(train.item.unique())
for user in train.user.unique():
    user = [user] * len(item)
    users.extend(user)
    items.extend(item)
    preds.extend(list(lightgcn.predict(user, item)))

all_predictions = pd.DataFrame(data={"user": users, "item":items, "prediction":preds})

merged = pd.merge(train, all_predictions, on=["user", "item"], how="outer")
all_predictions = merged[merged.label.isnull()].drop('label', axis=1)

In [78]:
all_predictions = all_predictions.rename(columns={"user":"userID", 'item':"itemID", "label":"rating", "song_by": "song"})
test = test.rename(columns={"user":"userID", 'item':"itemID", "label":"rating", "song_by": "song"})

In [83]:
scores = evaluate(
    model=lightgcn,
    data=test_data,
    neg_sampling=True,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)
scores

eval_listwise: 100%|██████████| 689/689 [00:01<00:00, 513.11it/s]


{'loss': 0.5726528606170258,
 'roc_auc': 0.9392017125025194,
 'precision': 0.036284470246734396,
 'recall': 0.06498846057989743,
 'ndcg': 0.1354837731009233}

#### MAP

It is the average precision for each user normalized over all users.

In [79]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
print(f"MAP @ {TOP_K}: {eval_map}")

MAP @ 10: 0.023336243271233598


#### NDCG

Normalized Discounted Cumulative Gain (NDCG) - evaluates how well the predicted items for a user are ranked based on relevance


In [84]:
eval_ndcg = scores['ndcg']
print(f"NDCG: {eval_ndcg}")

NDCG: 0.1354837731009233


In [80]:
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
print(f"NDCG @ {TOP_K}: {eval_ndcg}")

NDCG @ 10: 0.05353667894262494


#### Precision Recall

Precision - this measures the proportion of recommended items that are relevant

Recall - this measures the proportion of relevant items that are recommended

In [85]:
eval_precision = scores['precision']
eval_recall = scores['recall']
print(f"Precision: {eval_precision} \n Recall: {eval_recall}")

Precision: 0.036284470246734396 
 Recall: 0.06498846057989743


In [81]:
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
print(f"Precision @ {TOP_K}: {eval_precision} \n Recall @ {TOP_K}: {eval_recall}")

Precision @ 10: 0.0362844702467344 
 Recall @ 10: 0.06498846057989743
